In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torchvision

class ResNetMultilabel(nn.Module):
    """基於 ResNet 的 multi-label 分類模型"""
    def __init__(self, num_classes, pretrained=True, model_name='resnet50', dropout_rate=0.5):
        super(ResNetMultilabel, self).__init__()

        # 載入預訓練的 ResNet 變體
        if model_name == 'resnet18':
            backbone = torchvision.models.resnet18(pretrained=pretrained)
            num_features = 512
        elif model_name == 'resnet34':
            backbone = torchvision.models.resnet34(pretrained=pretrained)
            num_features = 512
        elif model_name == 'resnet50':
            backbone = torchvision.models.resnet50(pretrained=pretrained)
            num_features = 2048
        elif model_name == 'resnet101':
            backbone = torchvision.models.resnet101(pretrained=pretrained)
            num_features = 2048
        else:
            raise ValueError(f"不支援的模型: {model_name}")

        # 移除原本分類層，只保留到 avgpool
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        # 新增自訂的 multi-label 分類頭
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, num_classes)
        )

        print(f"模型初始化完成: {model_name}, 預訓練: {pretrained}")
        print(f"特徵維度: {num_features}, 類別數: {num_classes}")

    def forward(self, x):
        features = self.backbone(x)
        features = features.view(features.size(0), -1)  # Flatten
        output = self.classifier(features)
        return output


# === 1. 建立 test CSV（從 label 檔推得 image 名）===
def create_multilabel_csv(image_dir, label_dir, output_csv):
    data = []
    for label_file in os.listdir(label_dir):
        if not label_file.endswith('.txt'):
            continue
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as f:
            lines = f.readlines()
            if len(lines) == 0:
                continue
            classes = set()
            for line in lines:
                class_id = line.split()[0]
                classes.add(class_id)
        image_name = label_file.replace('.txt', '.jpg')  # 確認副檔名是否正確
        data.append({'image_name': image_name, 'labels': ' '.join(sorted(classes))})

    pd.DataFrame(data).to_csv(output_csv, index=False)
    print(f"✅ Test CSV saved to {output_csv}")

# === 2. 自訂 Dataset 類 ===
class MultiLabelDataset(Dataset):
    def __init__(self, csv_path, img_dir, num_classes, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.num_classes = num_classes
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image_name'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        labels = row['labels'].split()
        multi_hot = torch.zeros(self.num_classes, dtype=torch.float32)
        for l in labels:
            multi_hot[int(l)] = 1.0

        return image, multi_hot, row['image_name']

# === 3. 資料與模型設定 ===
num_classes = 7
test_img_dir = 'datasets/images/test'
test_label_dir = 'datasets/labels/test'
test_csv_path = 'ground_truth.csv'
create_multilabel_csv(test_img_dir, test_label_dir, test_csv_path)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 4. 載入模型 ===
model = ResNetMultilabel(
        num_classes=7,
        pretrained=True,
        model_name="resnet50",
        dropout_rate=0.5
    ).to("cuda")
model.load_state_dict(torch.load('outputs/resnet50_bs32_lr0.0005_ep30_20250531_145459/best_multilabel_model.pt'))
model.eval()
# === 5. 測試與評估 ===
from sklearn.metrics import precision_score, recall_score, f1_score

test_dataset = MultiLabelDataset(test_csv_path, test_img_dir, num_classes, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64)

threshold = 0.5
results = []

correct_per_class = torch.zeros(num_classes)
total_per_class = torch.zeros(num_classes)

# Micro-averaged 計數
total_correct = 0
total_preds = 0
total_labels = 0

# 每一筆的 y_true / y_pred（for macro 指標）
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels, img_names in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.sigmoid(outputs)
        preds = (probs > threshold).float()

        # 累積每一類的準確數
        correct_per_class += ((preds == labels) * labels).sum(dim=0).cpu()
        total_per_class += labels.sum(dim=0).cpu()

        # Micro-averaged 計數
        total_correct += (preds * labels).sum().item()
        total_preds += preds.sum().item()
        total_labels += labels.sum().item()

        # 收集所有 prediction & label（for macro）
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

        # 儲存預測結果（for 輸出 CSV）
        for name, pred in zip(img_names, preds.cpu()):
            label_indices = [str(i) for i, val in enumerate(pred) if val == 1]
            results.append({'image_name': name, 'labels': ' '.join(label_indices)})

# === 指標計算 ===
all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()

# Macro 指標
macro_precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
macro_recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

# Micro 指標
precision = total_correct / (total_preds + 1e-8)
recall = total_correct / (total_labels + 1e-8)
f1 = 2 * precision * recall / (precision + recall + 1e-8)

# Per-label Accuracy
per_label_accuracy = (correct_per_class / (total_per_class + 1e-8))

print("📊 Evaluation Results")
print(f"🎯 Micro:")
print(f"  - Precision: {precision:.4f}")
print(f"  - Recall:    {recall:.4f}")
print(f"  - F1-score:  {f1:.4f}")
print(f"\n🎯 Macro:")
print(f"  - Precision: {macro_precision:.4f}")
print(f"  - Recall:    {macro_recall:.4f}")
print(f"  - F1-score:  {macro_f1:.4f}")
print(f"\n🎯 Per-label Accuracy (mean): {per_label_accuracy.mean().item():.4f}\n")

for i, acc in enumerate(per_label_accuracy):
    print(f"  - Label {i}: Accuracy = {acc.item():.4f}")

# 儲存預測結果 CSV
output_csv = 'test_predictions.csv'
pd.DataFrame(results).to_csv(output_csv, index=False)
print(f"\n✅ Prediction CSV saved to {output_csv}")



✅ Test CSV saved to result.csv


/home/michael920403/anaconda3/envs/yolo_app/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/michael920403/anaconda3/envs/yolo_app/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


模型初始化完成: resnet50, 預訓練: True
特徵維度: 2048, 類別數: 7
📊 Evaluation Results
🎯 Micro:
  - Precision: 0.9849
  - Recall:    0.9477
  - F1-score:  0.9659

🎯 Macro:
  - Precision: 0.9868
  - Recall:    0.9397
  - F1-score:  0.9620

🎯 Per-label Accuracy (mean): 0.9397

  - Label 0: Accuracy = 0.9383
  - Label 1: Accuracy = 0.9333
  - Label 2: Accuracy = 0.9512
  - Label 3: Accuracy = 1.0000
  - Label 4: Accuracy = 0.9412
  - Label 5: Accuracy = 0.8500
  - Label 6: Accuracy = 0.9639

✅ Prediction CSV saved to test_predictions.csv
